In [1]:
import cv2, io, time, imutils, datetime, os, traceback, grpc
from imutils.video import VideoStream
from classes import imagenet_classes
import opencv_jupyter_ui as jcv2
import numpy as np

from ovmsclient import make_grpc_client
from tensorflow import make_tensor_proto, make_ndarray
from tensorflow_serving.apis import predict_pb2
from tensorflow_serving.apis import prediction_service_pb2_grpc

In [2]:
GRPC_URL = "localhost:9000"
RTSP_URL = "rtsp://localhost:8554/server"
WIDTH = 224
HEIGHT = 224
FPS = 10

In [3]:
grpc_client = make_grpc_client(GRPC_URL)
print("* gRPC Socket succesfully opened")

* gRPC Socket succesfully opened


In [4]:

try:
    rtsp_stream = VideoStream(RTSP_URL).start()
    print("* RTSP Stream succesfully opened")
except Exception as e:
    print("* RTSP Stream is unavailable, please close any other notebooks that might be accessing the stream first")

* RTSP Stream succesfully opened


In [5]:
def resnet(client, img):
    output = grpc_client.predict({"0": img}, "resnet")
    result_index = np.argmax(output[0])
    predicted_class = imagenet_classes[result_index]
    return predicted_class

In [ ]:
def prep_image(img, size = 224, rgb_image=True):
    img = cv2.resize(img, (WIDTH, HEIGHT))
    img = img.astype('float32')

    img = img[:, :, [2, 1, 0]]
    img = img.transpose(2,0,1).reshape(1,3, HEIGHT, WIDTH)
    return img


In [ ]:
def draw_label(img, text, pos, bg_color):
    font_face = cv2.FONT_HERSHEY_SIMPLEX
    scale = 0.4
    color = (0, 0, 0)
    thickness = cv2.FILLED
    margin = 2
    txt_size = cv2.getTextSize(text, font_face, scale, thickness)

    ##img = np.frombuffer(content, dtype=np.uint8)
   ## img = cv2.imdecode(img, cv2.IMREAD_COLOR)  # BGR format
    # retrived array has BGR format and 0-255 normalization
    # format of data is HWC
    # add image preprocessing if needed by the model
    end_x = pos[0] + txt_size[0][0] + margin
    end_y = pos[1] - txt_size[0][1] - margin
    
    cv2.rectangle(img, pos, (end_x, end_y), bg_color, thickness)
    cv2.putText(img, text, pos, font_face, scale, color, 1, cv2.LINE_AA)

In [ ]:
while True:
    try: 
        frame = rtsp_stream.read()
        try: 
            if frame is not None:
                aux_frame = prep_image(frame, WIDTH, HEIGHT)
                result = resnet(grpc_client, aux_frame)
                draw_label(frame, str(result), (20,20), (255,0,0))
                title = "Prediction OUTPUT"
                jcv2.imshow(title, frame)
            else: 
                continue 
                
            time.sleep(1/FPS)
        except Exception as e:
            print("error while running inference", e)
            print(traceback.format_exc())
          
    except Exception as e:
        print("error while grabbing RTSP stream", e)